In [37]:
import numpy as np
import sympy
import re

Core Testing

>Solution Function

>>Solution Function 1

In [9]:
def solve_linear_equations(coefficients, constants):
    try:
        # Trying to find a unique solution
        solution = np.linalg.solve(coefficients, constants)
    except np.linalg.LinAlgError:
        # Handle overdetermined systems
        solution, _, _, _ = np.linalg.lstsq(coefficients, constants, rcond=None)
    
    return solution


>>Solution Function 2

In [17]:
def solve_linear_equations(coefficients, constants):
    try:
        # Trying to find a unique solution
        solution = np.linalg.solve(coefficients, constants)
        return solution, "Unique solution"
    except np.linalg.LinAlgError as e:
        if 'Singular matrix' in str(e):
            # This means the matrix is singular and there's no unique solution
            # This can be due to no solutions or infinite solutions
            # We'll use lstsq to determine which one it is
            solution, residuals, rank, s = np.linalg.lstsq(coefficients, constants, rcond=None)
            if residuals.size == 0 or np.allclose(residuals, 0):
                return solution, "Infinite solutions"
            else:
                return solution, "No solution"
        else:
            # This is some other kind of linear algebra error we didn't expect
            raise

>>>Solution Function 3 (Handles Non-square Matrix)

In [23]:
def solve_linear_equations(coefficients, constants):
    # If matrix is not square, use lstsq to find the least squares solution
    if coefficients.shape[0] != coefficients.shape[1]:
        solution, residuals, rank, s = np.linalg.lstsq(coefficients, constants, rcond=None)
        if residuals.size == 0 or np.allclose(residuals, 0):
            return solution, "Infinite solutions"
        else:
            return solution, "No solution"
    else:
        try:
            solution = np.linalg.solve(coefficients, constants)
            return solution, "Unique solution"
        except np.linalg.LinAlgError as e:
            if 'Singular matrix' in str(e):
                return None, "No solution"
            else:
                raise

In [31]:
def solve_linear_equations(coefficients, constants):
    # Perform Gaussian elimination or row reduction to obtain RREF
    rref_matrix = np.hstack((coefficients, constants.reshape(-1, 1)))
    rref_matrix = sympy.Matrix(rref_matrix).rref()[0]
    rref_matrix = np.array(rref_matrix.tolist(), dtype=float)

    num_rows, num_cols = rref_matrix.shape
    free_vars = num_cols - num_rows - 1  # -1 for the constants column

    if free_vars > 0:
        # Infinite solutions, proceed to express in terms of free variables
        solution_set = []
        for i in range(num_rows):
            leading_coeff = rref_matrix[i][i]
            expression = f"{rref_matrix[i, -1]/leading_coeff}"
            for j in range(i + 1, num_cols - 1):
                if rref_matrix[i, j] != 0:
                    expression += f" - {rref_matrix[i, j]/leading_coeff}*x{j+1}"
            solution_set.append(expression)
        # Add the free variables
        for i in range(free_vars):
            solution_set.append(f"x{num_rows+i+1}")
        
        parametric_solution = ', '.join(solution_set)
        return parametric_solution, "Infinite solutions"
    else:
        # No free variables, system has a unique solution
        solution = np.linalg.solve(coefficients, constants)
        return solution, "Unique solution"




System Size Checking Function

In [ ]:
def check_system_size(num_equations, num_variables):
    if num_variables > 5 or num_equations > 5:
        raise ValueError("This system can only handle up to a 5x5 matrix.")
    if num_variables < 1 or num_equations < 1:
        raise ValueError("This system requires at least a 1x1 matrix.")


Solution Formating Function

In [41]:
def format_parametric_solution(solution_str):
    # Split the solution string into individual variable components
    variables = solution_str.split(',')

    # Define a pattern to match the variables like 'x4' and 'x5'
    var_pattern = re.compile(r'x\d+')

    formatted_solution = []

    for i, var in enumerate(variables):
        # Find all free variables in the expression
        free_vars = var_pattern.findall(var)

        # Correctly identify negative and positive floats
        expr = re.sub(r'(?<!\w)-?\d+\.\d+', lambda m: f"{float(m.group(0)):0.3f}", var)

        # If the variable part is a free variable, format it accordingly
        if i + 1 in [int(free_var[1:]) for free_var in free_vars]:
            formatted_solution.append(f"x{i+1} is a free variable")
        else:
            # Remove unnecessary pluses and correct double negatives
            expr = expr.replace(' - -', ' + ').replace('+-', '-')
            formatted_solution.append(f"x{i+1} = {expr.strip()}")

    return formatted_solution

Input Segment

In [24]:
num_variables = int(input("Enter the number of variables (up to 5): "))
num_equations = int(input("Enter the number of equations: "))

In [25]:
coefficients = []
constants = []

for i in range(num_equations):
    equation = input(f"Enter coefficients and constant for equation {i+1}, separated by space: ")
    values = [float(x) for x in equation.split()]
    coefficients.append(values[:-1])  # All but last
    constants.append(values[-1])      # Last element

In [19]:
coefficients = []
constants = []

for i in range(num_equations):
    # Ask the user to enter only the coefficients and constants as numbers separated by spaces
    equation = input(f"Enter the coefficients for equation {i+1} followed by the constant, separated by space: ")
    
    # Split the input string by spaces, convert each to a float, and append to the lists
    values = [float(x) for x in equation.split()]
    
    # Assuming the last number is the constant, and the rest are coefficients
    coefficients.append(values[:-1])  # All but last
    constants.append(values[-1])      # Last element


In [26]:
coefficients_array = np.array(coefficients)
constants_array = np.array(constants)

In [27]:
solution = solve_linear_equations(coefficients_array, constants_array)
print("Solution:", solution)

Solution: (array([ 0.16975854,  0.91372203, -0.21481154,  0.41740283,  0.14355124]), 'Infinite solutions')


Test

2x + 3y - z + w - 2v = 4

4x - y + 5z - 2w + v = -2

-x + 2y + 3z + w - 3v = 1

In [32]:
# Test:
coefficients_array = np.array([
    [2, 3, -1, 1, -2],
    [4, -1, 5, -2, 1],
    [-1, 2, 3, 1, -3]
])

constants_array = np.array([4, -2, 1])

In [35]:
solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)

Status: Infinite solutions


In [36]:
print("Solution:", solution)

Solution: 0.19047619047619047 - -0.27380952380952384*x4 - 0.32142857142857145*x5, 1.0952380952380953 - 0.4880952380952381*x4 - -0.9642857142857143*x5, -0.3333333333333333 - -0.08333333333333333*x4 - -0.25*x5, x4, x5


In [46]:
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


x1 = 0.190 + 0.274*x4 - 0.321*x5
x2 = 1.095 - 0.488*x4 + 0.964*x5
x3 = -0.333 + 0.083*x4 + 0.250*x5
x4 is a free variable
x5 is a free variable
